In [1]:
import pandas as pd
import requests

In [2]:
url = "https://api.worldbank.org/v2/country/all/indicator/NY.GDP.PCAP.PP.KD?format=json&per_page=20000"
response = requests.get(url)
data = response.json()

In [3]:
data[1][0]

{'indicator': {'id': 'NY.GDP.PCAP.PP.KD',
  'value': 'GDP per capita, PPP (constant 2021 international $)'},
 'country': {'id': 'ZH', 'value': 'Africa Eastern and Southern'},
 'countryiso3code': 'AFE',
 'date': '2024',
 'value': 3968.96375122681,
 'unit': '',
 'obs_status': '',
 'decimal': 0}

In [4]:
gdp_df = pd.DataFrame(data[1])
gdp_df = gdp_df[['countryiso3code','country','date','value']]
gdp_df.rename(columns={'value':'gdp_per_capita_ppp','countryiso3code':'country_code'}, inplace=True)

In [5]:
country_df = pd.json_normalize(gdp_df['country'])

In [6]:
gdp_df['country'] = country_df['value']

In [7]:
gdp_df

,country_code,country,date,gdp_per_capita_ppp
0,AFE,Africa Eastern and Southern,2024,3968.963751
1,AFE,Africa Eastern and Southern,2023,3948.142721
2,AFE,Africa Eastern and Southern,2022,3974.244214
3,AFE,Africa Eastern and Southern,2021,3933.580905
4,AFE,Africa Eastern and Southern,2020,3861.068816
...,...,...,...,...
17285,ZWE,Zimbabwe,1964,NaN
17286,ZWE,Zimbabwe,1963,NaN
17287,ZWE,Zimbabwe,1962,NaN
17288,ZWE,Zimbabwe,1961,NaN


# Q2 .Data for Life expectancy

In [8]:
url2 = "https://api.worldbank.org/v2/country/all/indicator/SP.DYN.LE00.IN?format=json&per_page=20000"
response2 = requests.get(url2)
data2 = response2.json()

In [9]:
data2[1][0]

{'indicator': {'id': 'SP.DYN.LE00.IN',
  'value': 'Life expectancy at birth, total (years)'},
 'country': {'id': 'ZH', 'value': 'Africa Eastern and Southern'},
 'countryiso3code': 'AFE',
 'date': '2024',
 'value': None,
 'unit': '',
 'obs_status': '',
 'decimal': 0}

In [10]:
life_df = pd.DataFrame(data2[1])

In [11]:
life_df = life_df[['countryiso3code','country','date','value']]
life_df.rename(columns={'value':'Life_Expectancy','countryiso3code':'country_code'}, inplace=True)

In [12]:
country1_df = pd.json_normalize(life_df['country'])

In [13]:
life_df['country'] = country1_df['value']

In [14]:
life_df

,country_code,country,date,Life_Expectancy
0,AFE,Africa Eastern and Southern,2024,NaN
1,AFE,Africa Eastern and Southern,2023,65.146291
2,AFE,Africa Eastern and Southern,2022,64.487020
3,AFE,Africa Eastern and Southern,2021,62.979999
4,AFE,Africa Eastern and Southern,2020,63.766484
...,...,...,...,...
17285,ZWE,Zimbabwe,1964,55.431000
17286,ZWE,Zimbabwe,1963,54.942000
17287,ZWE,Zimbabwe,1962,54.453000
17288,ZWE,Zimbabwe,1961,53.966000


# Merge gdp_df and life_df dataframe

In [15]:
df= pd.merge(gdp_df,life_df, on =["country_code","country","date"],how ="inner")

In [16]:
df

,country_code,country,date,gdp_per_capita_ppp,Life_Expectancy
0,AFE,Africa Eastern and Southern,2024,3968.963751,NaN
1,AFE,Africa Eastern and Southern,2023,3948.142721,65.146291
2,AFE,Africa Eastern and Southern,2022,3974.244214,64.487020
3,AFE,Africa Eastern and Southern,2021,3933.580905,62.979999
4,AFE,Africa Eastern and Southern,2020,3861.068816,63.766484
...,...,...,...,...,...
17285,ZWE,Zimbabwe,1964,NaN,55.431000
17286,ZWE,Zimbabwe,1963,NaN,54.942000
17287,ZWE,Zimbabwe,1962,NaN,54.453000
17288,ZWE,Zimbabwe,1961,NaN,53.966000


##Q4.You can also get more information about the available countries (region, capital city, income level classification, etc.) by using the Country API. Use this API to pull in all available data. Merge this with your other datasets. Use this to now remove the rows that correspond to regions and not countries.

In [17]:
country= "https://api.worldbank.org/v2/country?format=json&per_page=1000"
response = requests.get(country)
country_data = response.json()

In [18]:
response.status_code

200

In [19]:
countries_df = pd.json_normalize(country_data[1])

In [20]:
countries_df = countries_df[['id','name','region.value','capitalCity','incomeLevel.value','lendingType.value']]

In [21]:
countries_df.rename(columns={'region.value': 'region','incomeLevel.value': 'incomeLevel','lendingType.value': 'lendingType'}, inplace=True)

In [22]:
countries_df

,id,name,region,capitalCity,incomeLevel,lendingType
0,ABW,Aruba,Latin America & Caribbean,Oranjestad,High income,Not classified
1,AFE,Africa Eastern and Southern,Aggregates,,Aggregates,Aggregates
2,AFG,Afghanistan,"Middle East, North Africa, Afghanistan & Pakistan",Kabul,Low income,IDA
3,AFR,Africa,Aggregates,,Aggregates,Aggregates
4,AFW,Africa Western and Central,Aggregates,,Aggregates,Aggregates
...,...,...,...,...,...,...
291,XZN,Sub-Saharan Africa excluding South Africa and ...,Aggregates,,Aggregates,Aggregates
292,YEM,"Yemen, Rep.","Middle East, North Africa, Afghanistan & Pakistan",Sana'a,Low income,IDA
293,ZAF,South Africa,Sub-Saharan Africa,Pretoria,Upper middle income,IBRD
294,ZMB,Zambia,Sub-Saharan Africa,Lusaka,Lower middle income,IDA


In [23]:
df = df.merge(countries_df, left_on='country_code', right_on='id', how='left')

In [24]:
df

,country_code,country,date,gdp_per_capita_ppp,Life_Expectancy,id,name,region,capitalCity,incomeLevel,lendingType
0,AFE,Africa Eastern and Southern,2024,3968.963751,NaN,AFE,Africa Eastern and Southern,Aggregates,,Aggregates,Aggregates
1,AFE,Africa Eastern and Southern,2023,3948.142721,65.146291,AFE,Africa Eastern and Southern,Aggregates,,Aggregates,Aggregates
2,AFE,Africa Eastern and Southern,2022,3974.244214,64.487020,AFE,Africa Eastern and Southern,Aggregates,,Aggregates,Aggregates
3,AFE,Africa Eastern and Southern,2021,3933.580905,62.979999,AFE,Africa Eastern and Southern,Aggregates,,Aggregates,Aggregates
4,AFE,Africa Eastern and Southern,2020,3861.068816,63.766484,AFE,Africa Eastern and Southern,Aggregates,,Aggregates,Aggregates
...,...,...,...,...,...,...,...,...,...,...,...
17285,ZWE,Zimbabwe,1964,NaN,55.431000,ZWE,Zimbabwe,Sub-Saharan Africa,Harare,Lower middle income,Blend
17286,ZWE,Zimbabwe,1963,NaN,54.942000,ZWE,Zimbabwe,Sub-Saharan Africa,Harare,Lower middle income,Blend
17287,ZWE,Zimbabwe,1962,NaN,54.453000,ZWE,Zimbabwe,Sub-Saharan Africa,Harare,Lower middle income,Blend
17288,ZWE,Zimbabwe,1961,NaN,53.966000,ZWE,Zimbabwe,Sub-Saharan Africa,Harare,Lower middle income,Blend


# Bonus

###Q1.Adjust your request so that it returns data just for the United States.

In [25]:
url3 = "https://api.worldbank.org/v2/country/USA/indicator/NY.GDP.PCAP.PP.KD?format=json&per_page=500"
response3 = requests.get(url3)
data3 = response3.json()

In [26]:
response.status_code

200

In [27]:
USA_gdp_df = pd.DataFrame(data3[1])
USA_gdp_df = USA_gdp_df[['countryiso3code','country','date','value']]
USA_gdp_df.rename(columns={'value':'gdp_per_capita_ppp','countryiso3code':'country_code'}, inplace=True)

In [28]:
country2_df = pd.json_normalize(USA_gdp_df['country'])

In [29]:
USA_gdp_df['country'] = country2_df['value']

In [30]:
USA_gdp_df

,country_code,country,date,gdp_per_capita_ppp
0,USA,United States,2024,75491.608435
1,USA,United States,2023,74158.716417
2,USA,United States,2022,72679.257540
3,USA,United States,2021,71307.401728
4,USA,United States,2020,67342.070928
...,...,...,...,...
60,USA,United States,1964,NaN
61,USA,United States,1963,NaN
62,USA,United States,1962,NaN
63,USA,United States,1961,NaN


In [31]:
url4 = "https://api.worldbank.org/v2/country/USA/indicator/SP.DYN.LE00.IN?format=json&per_page=20000"
response4 = requests.get(url4)
data4 = response4.json()

In [32]:
USA_life_df = pd.DataFrame(data4[1])

In [33]:
USA_life_df = USA_life_df[['countryiso3code','country','date','value']]
USA_life_df.rename(columns={'value':'Life_Expectancy','countryiso3code':'country_code'}, inplace=True)

In [34]:
country3_df = pd.json_normalize(USA_life_df['country'])

In [35]:
USA_life_df['country'] = country3_df['value']

In [36]:
USA_life_df

,country_code,country,date,Life_Expectancy
0,USA,United States,2024,NaN
1,USA,United States,2023,78.385366
2,USA,United States,2022,77.434146
3,USA,United States,2021,76.329268
4,USA,United States,2020,76.980488
...,...,...,...,...
60,USA,United States,1964,70.165854
61,USA,United States,1963,69.917073
62,USA,United States,1962,70.119512
63,USA,United States,1961,70.270732


In [37]:
USA_df= pd.merge(USA_gdp_df,USA_life_df, on =["country_code","country","date"],how ="inner")

In [38]:
USA_df

,country_code,country,date,gdp_per_capita_ppp,Life_Expectancy
0,USA,United States,2024,75491.608435,NaN
1,USA,United States,2023,74158.716417,78.385366
2,USA,United States,2022,72679.257540,77.434146
3,USA,United States,2021,71307.401728,76.329268
4,USA,United States,2020,67342.070928,76.980488
...,...,...,...,...,...
60,USA,United States,1964,NaN,70.165854
61,USA,United States,1963,NaN,69.917073
62,USA,United States,1962,NaN,70.119512
63,USA,United States,1961,NaN,70.270732


In [39]:
USA_df = USA_df.merge(countries_df, left_on='country_code', right_on='id', how='left')

In [40]:
USA_df

,country_code,country,date,gdp_per_capita_ppp,Life_Expectancy,id,name,region,capitalCity,incomeLevel,lendingType
0,USA,United States,2024,75491.608435,NaN,USA,United States,North America,Washington D.C.,High income,Not classified
1,USA,United States,2023,74158.716417,78.385366,USA,United States,North America,Washington D.C.,High income,Not classified
2,USA,United States,2022,72679.257540,77.434146,USA,United States,North America,Washington D.C.,High income,Not classified
3,USA,United States,2021,71307.401728,76.329268,USA,United States,North America,Washington D.C.,High income,Not classified
4,USA,United States,2020,67342.070928,76.980488,USA,United States,North America,Washington D.C.,High income,Not classified
...,...,...,...,...,...,...,...,...,...,...,...
60,USA,United States,1964,NaN,70.165854,USA,United States,North America,Washington D.C.,High income,Not classified
61,USA,United States,1963,NaN,69.917073,USA,United States,North America,Washington D.C.,High income,Not classified
62,USA,United States,1962,NaN,70.119512,USA,United States,North America,Washington D.C.,High income,Not classified
63,USA,United States,1961,NaN,70.270732,USA,United States,North America,Washington D.C.,High income,Not classified


# Q2-Adjust your request so that it returns data just for the United States for the year 2021.

In [41]:
url5 = "https://api.worldbank.org/v2/country/USA/indicator/NY.GDP.PCAP.PP.KD?format=json&date=2021"
response5 = requests.get(url5)
data5 = response5.json()

In [42]:
USA_gdp_2021_df = pd.DataFrame(data5[1])

In [43]:
USA_gdp_2021_df['country'] = USA_gdp_2021_df['country'].str['value']

In [44]:
USA_gdp_2021_df = USA_gdp_2021_df[['countryiso3code', 'country', 'date', 'value']]

In [45]:
USA_gdp_2021_df.rename(columns={'countryiso3code': 'country_code','value': 'Gdp_per_capita_ppp'}, inplace=True)
USA_gdp_2021_df

,country_code,country,date,Gdp_per_capita_ppp
0,USA,United States,2021,71307.401728


In [46]:
url6 = "https://api.worldbank.org/v2/country/USA/indicator/SP.DYN.LE00.IN?format=json&date=2021"
response6 = requests.get(url6)
data6 = response6.json()

In [47]:
USA_life_2021_df = pd.DataFrame(data6[1])

In [48]:
USA_life_2021_df['country'] = USA_life_2021_df['country'].str['value']

In [49]:
USA_life_2021_df = USA_life_2021_df[['countryiso3code', 'country', 'date', 'value']]

In [50]:
USA_life_2021_df.rename(columns={'countryiso3code': 'country_code','value': 'Life_expectancy'}, inplace=True)
USA_life_2021_df

,country_code,country,date,Life_expectancy
0,USA,United States,2021,76.329268


In [51]:
USA_2021_df= pd.merge(USA_gdp_2021_df,USA_life_2021_df, on =["country_code","country","date"],how ="inner")

In [52]:
USA_2021_df

,country_code,country,date,Gdp_per_capita_ppp,Life_expectancy
0,USA,United States,2021,71307.401728,76.329268


In [53]:
USA_2021_df = USA_2021_df.merge(countries_df, left_on='country_code', right_on='id', how='left')

In [54]:
USA_2021_df = USA_2021_df.drop(columns=['id', 'name'])

In [55]:
USA_2021_df

,country_code,country,date,Gdp_per_capita_ppp,Life_expectancy,region,capitalCity,incomeLevel,lendingType
0,USA,United States,2021,71307.401728,76.329268,North America,Washington D.C.,High income,Not classified


###  Q.3-Adjust your request so that it returns data just for the United States for the years 2000 through 2021.



In [56]:
url7 = "https://api.worldbank.org/v2/country/USA/indicator/NY.GDP.PCAP.PP.KD?format=json&date=2000:2021"
response7 = requests.get(url7)
data7 = response7.json()
USA_gdp_2000_2021_df = pd.DataFrame(data7[1])

In [57]:
USA_gdp_2000_2021_df['country'] = USA_gdp_2000_2021_df['country'].str['value']

In [58]:
USA_gdp_2000_2021_df = USA_gdp_2000_2021_df[['countryiso3code', 'country', 'date', 'value']]

In [84]:
USA_gdp_2000_2021_df.rename(columns={'countryiso3code': 'country_code','value': 'gdp_per_capita_ppp'}, inplace=True)
USA_gdp_2000_2021_df.head(2)

,country_code,country,date,gdp_per_capita_ppp
0,USA,United States,2021,71307.401728
1,USA,United States,2020,67342.070928


In [60]:
url8 = "https://api.worldbank.org/v2/country/USA/indicator/SP.DYN.LE00.IN?format=json&date=2000:2021"
response8 = requests.get(url8)
data8 = response8.json()
USA_life_2000_2021_df = pd.DataFrame(data8[1])

In [61]:
USA_life_2000_2021_df['country'] = USA_life_2000_2021_df['country'].str['value']

In [62]:
USA_life_2000_2021_df = USA_life_2000_2021_df[['countryiso3code', 'country', 'date', 'value']]

In [63]:
USA_life_2000_2021_df.rename(columns={'countryiso3code': 'country_code','value': 'life_expectancy'}, inplace=True)
USA_life_2000_2021_df.head()

,country_code,country,date,life_expectancy
0,USA,United States,2021,76.329268
1,USA,United States,2020,76.980488
2,USA,United States,2019,78.787805
3,USA,United States,2018,78.639024
4,USA,United States,2017,78.539024


In [64]:
USA_2000_2021_df= pd.merge(USA_gdp_2000_2021_df,USA_life_2000_2021_df, on =["country_code","country","date"],how ="inner")

In [65]:
USA_2000_2021_df = USA_2000_2021_df.merge(countries_df, left_on='country_code', right_on='id', how='left')

In [66]:
USA_2000_2021_df = USA_2000_2021_df.drop(columns=['id', 'name'])

In [86]:
USA_2000_2021_df.head(2)

,country_code,country,date,gdp_per_capita_ppp,life_expectancy,region,capitalCity,incomeLevel,lendingType
0,USA,United States,2021,71307.401728,76.329268,North America,Washington D.C.,High income,Not classified
1,USA,United States,2020,67342.070928,76.980488,North America,Washington D.C.,High income,Not classified


## Q4.Adjust your request so that it returns data for the United States and Canada for the years 2000 through 2021.

In [67]:
url_gdp = "https://api.worldbank.org/v2/country/USA;CAN/indicator/NY.GDP.PCAP.PP.KD?format=json&date=2000:2021"
response_gdp = requests.get(url_gdp)
data_gdp = response_gdp.json()

In [68]:
gdp_usa_canada_df = pd.DataFrame(data_gdp[1])

In [69]:
gdp_usa_canada_df['country'] = gdp_usa_canada_df['country'].str['value']

In [70]:
gdp_usa_canada_df = gdp_usa_canada_df[['countryiso3code', 'country', 'date', 'value']]

In [71]:
gdp_usa_canada_df.rename(columns={'countryiso3code': 'country_code','value': 'gdp_per_capita_ppp'}, inplace=True)
gdp_usa_canada_df.head(2)

,country_code,country,date,gdp_per_capita_ppp
0,CAN,Canada,2021,56995.117678
1,CAN,Canada,2020,54092.879032


In [72]:
url_life = "https://api.worldbank.org/v2/country/USA;CAN/indicator/SP.DYN.LE00.IN?format=json&date=2000:2021&per_page=500"
response_life = requests.get(url_life)
data_life = response_life.json()

In [73]:
life_usa_canada_df = pd.DataFrame(data_life[1])


In [74]:
life_usa_canada_df['country'] = life_usa_canada_df['country'].str['value']

In [75]:
life_usa_canada_df = life_usa_canada_df[['countryiso3code', 'country', 'date', 'value']]

In [76]:
life_usa_canada_df.rename(columns={'countryiso3code': 'country_code','value': 'life_expectancy'}, inplace=True)
life_usa_canada_df.head(2)

,country_code,country,date,life_expectancy
0,CAN,Canada,2021,81.516829
1,CAN,Canada,2020,81.535122


In [77]:
usa_canada_df = pd.merge(gdp_usa_canada_df,life_usa_canada_df,on=['country_code', 'country', 'date'],how='inner')
usa_canada_df.head()

,country_code,country,date,gdp_per_capita_ppp,life_expectancy
0,CAN,Canada,2021,56995.117678,81.516829
1,CAN,Canada,2020,54092.879032,81.535122
2,CAN,Canada,2019,57583.845798,82.163659
3,CAN,Canada,2018,57337.490910,81.808780
4,CAN,Canada,2017,56612.328537,81.812927


In [78]:
usa_canada_df = usa_canada_df.merge(countries_df,left_on='country_code',right_on='id',how='left')

In [81]:
usa_canada_df = usa_canada_df.drop(columns=['id', 'name'])

In [82]:
usa_canada_df.head()

,country_code,country,date,gdp_per_capita_ppp,life_expectancy,region,capitalCity,incomeLevel,lendingType
0,CAN,Canada,2021,56995.117678,81.516829,North America,Ottawa,High income,Not classified
1,CAN,Canada,2020,54092.879032,81.535122,North America,Ottawa,High income,Not classified
2,CAN,Canada,2019,57583.845798,82.163659,North America,Ottawa,High income,Not classified
3,CAN,Canada,2018,57337.490910,81.808780,North America,Ottawa,High income,Not classified
4,CAN,Canada,2017,56612.328537,81.812927,North America,Ottawa,High income,Not classified


## Q5.If you haven't already done so and you would like to get some additional practice using loops, use the page parameter in order to pull all records. Do not change the value of the per_page parameter. You will likely need to utilize a loop of some kind in order to pull all records.